In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Dataset, evaluate,Reader,SVD,SVDpp
from surprise import SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic
from surprise import KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise.model_selection import cross_validate
from surprise import KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
from collections import defaultdict
np.random.seed(20)

ModuleNotFoundError: No module named 'surprise'

In [2]:
#importing the song dataset
songData = pd.read_csv('Dataset For Song Recommendation System/song_data.csv') 
songData.head(4)

,user_id,song_id,listen_count,title,artist,song
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Jack Johnson,The Cove - Jack Johnson
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Paco De Lucia,Entre Dos Aguas - Paco De Lucia
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Kanye West,Stronger - Kanye West
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson,Constellations - Jack Johnson


In [26]:
#Function to find top N popular songs
def TopNSongsPopularityWise(songdata,Count):
    #Grouping according to song_id and finding the total listen count 
    dfSongId = songdata.groupby(['song_id','title']).agg({'listen_count': 'count'}) 
    #Sorting the total listen count in descending order
    dfSongId = dfSongId.sort_values('listen_count', ascending = False).reset_index()
    topSongs = list(dfSongId['title'])
    print('Top {} songs based on Popularity are:'.format(Count))
    for i in range(0,Count):
        print(topSongs[i])
TopNSongsPopularityWise(songData,10)

Top 10 songs based on Popularity are:
Sehr kosmisch
Undo
You\'re The One
Dog Days Are Over (Radio Edit)
Revelry
Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)
Secrets
Tive Sim
Fireflies
Hey_ Soul Sister


In [35]:
#Function to Find Songs Based on Content
def TopNSongsContentBased(songdata,title,Count):
    songdata.drop_duplicates(subset = 'title',keep ='first',inplace = True)
    songdata.reset_index(drop=True, inplace=True)
    songdata['Details'] = songdata['artist'] + songdata['title'] + songdata['song']
    #Creating a TF-IDF matrix
    tf = TfidfVectorizer(analyzer='word',ngram_range=(1,2),min_df=0,stop_words='english')
    tfidf_matrix = tf.fit_transform(songdata['Details'])
    #Finding the Cosine similarity Between the Song Details
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    titles  = songdata['title']  
    indices = pd.Series(songdata.index, index=songdata['title'])
    #Function to get N similar titles
    def get_recommnedation(title,cnt):
        similarSongIndices = []    
        i = indices[title]        
        simulatedScores = enumerate(cosine_sim[i])
        simulatedScores = sorted(simulatedScores, key = lambda x: x[1],reverse = True)
        #print(simulated_scores)
        count = -1
        for i in simulatedScores:
            count = count+1
            similarSongIndices.append(i[0])
            if count == cnt:
                break
        return similarSongIndices
    similarSongIndices = get_recommnedation(title,Count) 
    print('Top',Count,'Recommedations Based on Content for the Song',title)
    for i in range(1,len(similarSongIndices)):
        print(titles[similarSongIndices[i]])
TopNSongsContentBased(songData,'Come To Me',10)

Top 10 Recommedations Based on Content for the Song Come To Me
Undo
Isobel
Unravel
Army of Me
Cover Me
Come Back to Me
Hidden Place
Come & Talk To Me
Possibly Maybe
Human Behaviour


In [28]:
#This Function Gives Rating for the song according to popularity. Since this rating doesn't depend on the user's taste, idea was dropped
'''dfSongId = songData.groupby(['song_id','title']).agg({'listen_count': 'count'})
dfSongId = dfSongId.sort_values('listen_count', ascending = False).reset_index()
dfSongId['listen_count'] = dfSongId['listen_count']*100/(dfSongId['listen_count'].sum()) 
def find_rating(x):
    if(x>dfSongId['listen_count'][2000]):
        return 5
    elif(dfSongId['listen_count'][2000]>=x>dfSongId['listen_count'][4000]):
        return 4
    elif(dfSongId['listen_count'][4000]>=x>dfSongId['listen_count'][6000]):
        return 3
    elif(dfSongId['listen_count'][6000]>=x>dfSongId['listen_count'][8000]):
        return 2
    elif(dfSongId['listen_count'][8000]>=x>=dfSongId['listen_count'][9999]):
        return 1
    
    
dfSongId['popularity_rating'] = dfSongId.apply(lambda x: find_rating(x['listen_count']), axis=1)
dfSongId.drop(['listen_count'],axis=1,inplace=True)
result = pd.merge(songData, dfSongId, how='inner', on=['song_id','title'])'''

"dfSongId = songData.groupby(['song_id','title']).agg({'listen_count': 'count'})\ndfSongId = dfSongId.sort_values('listen_count', ascending = False).reset_index()\ndfSongId['listen_count'] = dfSongId['listen_count']*100/(dfSongId['listen_count'].sum()) \ndef find_rating(x):\n    if(x>dfSongId['listen_count'][2000]):\n        return 5\n    elif(dfSongId['listen_count'][2000]>=x>dfSongId['listen_count'][4000]):\n        return 4\n    elif(dfSongId['listen_count'][4000]>=x>dfSongId['listen_count'][6000]):\n        return 3\n    elif(dfSongId['listen_count'][6000]>=x>dfSongId['listen_count'][8000]):\n        return 2\n    elif(dfSongId['listen_count'][8000]>=x>=dfSongId['listen_count'][9999]):\n        return 1\n    \n    \ndfSongId['popularity_rating'] = dfSongId.apply(lambda x: find_rating(x['listen_count']), axis=1)\ndfSongId.drop(['listen_count'],axis=1,inplace=True)\nresult = pd.merge(songData, dfSongId, how='inner', on=['song_id','title'])"

In [29]:
#Song Data is grouped According to user id and total listen count for the particular user is obtained
listenCountAggDf = songData.groupby('user_id').agg({'listen_count': 'count'})
listenCountAggDf = listenCountAggDf[listenCountAggDf['listen_count']>10]
UsersSongDataDf = pd.merge(songData, listenCountAggDf, how='inner', on=['user_id'])
UsersSongDataDf = UsersSongDataDf.dropna()

In [30]:
#Finding the rating given by the user Explicitly by Taking the percentage
def SongRating(x,y):
    res = (x/y)*100
    if(res <=4):
        return 1 
    elif (4<res<=6):
        return 2
    elif(6<res<=8):
        return 3
    elif(8<res<=11):
        return 4
    elif(11<res):
        return 5

UsersSongDataDf['UserSongRating'] = UsersSongDataDf.apply(lambda x: SongRating(x['listen_count_x'],x['listen_count_y']), axis=1)
UsersSongDataDf.head(4)

,user_id,song_id,listen_count_x,title,artist,song,Details,listen_count_y,UserSongRating
0,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOAKIMP12A8C130995,1,The Cove,Jack Johnson,The Cove - Jack Johnson,Jack JohnsonThe CoveThe Cove - Jack Johnson,45,1
1,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Paco De Lucia,Entre Dos Aguas - Paco De Lucia,Paco De LuciaEntre Dos AguasEntre Dos Aguas - ...,45,2
2,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBXHDL12A81C204C0,1,Stronger,Kanye West,Stronger - Kanye West,Kanye WestStrongerStronger - Kanye West,45,1
3,b80344d063b5ccb3212f76538f3d9e43d87dca9e,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson,Constellations - Jack Johnson,Jack JohnsonConstellationsConstellations - Jac...,45,1


In [31]:
#Taking 20000 datapoints for Cross Validation and Finding the least error
UsersSongDatadf = UsersSongDataDf[0:20000]
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(UsersSongDatadf[['user_id', 'title', 'UserSongRating']], reader)
benchmark = []
algorithms = [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), 
              KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]
# Iterate over all algorithms
for algorithm in algorithms:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)
    
pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')    

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


,test_rmse,fit_time,test_time
Algorithm,,,
SVDpp,1.167422,1.334833,0.016487
SVD,1.188725,0.297853,0.018758
KNNBaseline,1.245916,0.016024,0.015641
BaselineOnly,1.251071,0.015920,0.010724
SlopeOne,1.460454,0.199212,0.016245
KNNWithMeans,1.460597,0.004811,0.016998
KNNBasic,1.460794,0.005207,0.014526
CoClustering,1.460919,0.519630,0.011369
NMF,1.460989,0.648878,0.016749


In [40]:
# Training the SVD algorithm with trainset
data = Dataset.load_from_df(UsersSongDataDf[['user_id', 'title', 'UserSongRating']], reader)
trainset, testset = train_test_split(data, test_size=0.2)
algo = SVD()

# Train the algorithm on the trainset, and predict ratings for the testset
algo.fit(trainset)
predictions = algo.test(testset)

# Then compute RMSE
print('The Root Mean Squared Error is',accuracy.rmse(predictions))

RMSE: 1.2120
The Root Mean Squared Error is 1.212012900893959


In [41]:
#Function to print recommended song to users
def get_top_n(predictions, n=10):    
    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n
top_n = get_top_n(predictions, n=4)
print('---------------USER ID-----------------------------------------------------SONGS-------------------------')
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

---------------USER ID-----------------------------------------------------SONGS-------------------------
f694565a4451ed33e6741ede8cd3a7eaad66bb84 ['I Get Around', 're:stacks', 'Girl With One Eye (Bayou Percussion Version)']
b943fa6a804abef677c48c67553bb8a1679ceae8 ['Gitana', "She\\'ll Come Back To Me", 'Mr. P-Mosh', 'War Pigs']
2e37aa820994c7e8465721a6a6ee78f48fc4df7a ['The Trance Is The Motion [Live]', 'Reach Out', 'Back To School (Mini Maggit) (LP Version)', 'Inolvidable']
2475b003df9c084c1488b2e63666e8cc87112180 ['Il tempio interiore', "Lady D\\'Arbanville", 'The Lost Get Found', 'Electricity']
645a5b400d31ce69611137da3ebd35d11d4dda4a ['Lights Of Ayodhya', 'Combat', 'The Perfect Drug', 'Crank That (Soulja Boy)']
e006b1a48f466bf59feefed32bec6494495a4436 ['Undo', 'Secrets', 'Horn Concerto No. 4 in E flat K495: II. Romance (Andante cantabile)', 'Cry For Help (Album Version)']
4b73abafd2a8309c29695ce368bbf17ab32f3d3a ['Balada Del Diablo y La Muerte', 'Otherside (Album Version)']
7c8338